In [2]:

import sympy as sp

sMul = sp.Mul

from symqups import s as CahillGlauberS
from symqups import MP_CONFIG
from symqups._internal.multiprocessing import mp_helper
from symqups.objects.scalars import (q, p, alpha, alphaD, W, mu, _Primed, 
                                     _DerivativeSymbol, hbar, StateFunction)
from symqups.objects.operators import qOp, pOp, annihilateOp, createOp, rho, Operator
from symqups.manipulations import define, qp2a, explicit, express, normal_ordered_equivalent, _deprime
from symqups.ordering import sOrdering
from symqups.quantization import s_quantize, naive_quantize
from symqups.cg import CG_transform, iCG_transform
from symqups.objects import scalars
from symqups._internal.cache import sub_cache, op2sc_subs_dict, sc2op_subs_dict, qp2alpha_subs_dict, alpha2qp_subs_dict
from symqups.star_product import Bopp, Star, _replace_diff
from symqups._internal.basic_routines import operation_routine
from symqups._internal.operator_handling import (
    separate_operator,
    separate_term_by_polynomiality,
    separate_term_oper_by_sub,
    collect_alpha_type_oper_from_monomial_by_sub,
    get_oper_sub
)
from symqups._internal.warnings import suppress_warning_repeats
from symqups._internal.grouping import UnBoppable, NotAnOperator, PhaseSpaceObject
from symqups.utils import get_random_poly
from symqups.eom import LindbladMasterEquation
from typing import Sequence

a = [alpha(i) for i in range(10)]
aop = [annihilateOp(i) for i in range(10)]
ad = [alphaD(i) for i in range(10)]
adop = [createOp(i) for i in range(10)]
x = sp.Symbol("x")
qq = [q(i) for i in range(10)]
pp = [p(i) for i in range(10)]
W = scalars.W
CahillGlauberS.val = sp.Number(0)
scalars.mu = 1
scalars.hbar = 1

CahillGlauberS

ImportError: cannot import name 'define' from 'symqups.manipulations' (/home/hendry/Documents/github_repo/SymQuPS/symqups/manipulations.py)

In [22]:
import time
start = time.time()
expr = qq[0]
expr.subs(qp2alpha_subs_dict)
print(time.time()-start)

0.0016405582427978516


In [21]:
import time
start = time.time()
expr = qq[0]
expr.subs({key : qp2alpha_subs_dict[key] for key in expr.atoms() & qp2alpha_subs_dict.keys()})
print(time.time()-start)

0.0008308887481689453


{q_{0}}

In [ ]:
expr = _Primed(sp.Derivative(a[0]*sp.Derivative(W, a[0]), a[0], a[1]))

\partial_{{\alpha_{0}}'}*\partial_{{\alpha_{1}}'}*{\alpha_{0}}'*\partial_{{\alpha_{0}}'}*StateFunction(t_{}, {\alpha_{0}}', {\overline{\alpha}_{0}}', {\alpha_{1}}', {\overline{\alpha}_{1}}', {\alpha_{2}}', {\overline{\alpha}_{2}}', {\alpha_{3}}', {\overline{\alpha}_{3}}', {\alpha_{4}}', {\overline{\alpha}_{4}}', {\alpha_{5}}', {\overline{\alpha}_{5}}', {\alpha_{6}}', {\overline{\alpha}_{6}}', {\alpha_{7}}', {\overline{\alpha}_{7}}', {\alpha_{8}}', {\overline{\alpha}_{8}}', {\alpha_{9}}', {\overline{\alpha}_{9}}')

In [60]:
def ddx(x):
    return _DerivativeSymbol(_Primed(x))

ddx(qq[0])

\partial_{{q_{0}}'}